In [7]:
import os
from dotenv import load_dotenv
from pathlib import Path # Use pathlib for robust path management
env_path = Path('../../') / '.env'

# 2. Load the environment variables from the root .env file
load_dotenv(dotenv_path=env_path)

# 3. Test that it loaded correctly
api_key = os.getenv("ANTHROPIC_API_KEY")

if api_key:
    print("API Key loaded successfully!")
    # Now you can use it to initialize your Anthropic client
    # anthropic_client = anthropic.Anthropic(api_key=api_key)
else:
    print("Error: ANTHROPIC_API_KEY not found. Check your .env file path.")

API Key loaded successfully!


In [8]:
import anthropic
import googlemaps
from datetime import datetime
import os
import json # Will use to format the output

# Initialize clients (assuming API key loaded)
api_key = os.getenv("ANTHROPIC_API_KEY")
client = anthropic.Anthropic(api_key=api_key)

# Mock Input Data
schedule_a = [
    "Friday 11/15: 5:00 PM - 7:00 PM (Gym)",
    "Saturday 11/16: 9:00 AM - 12:00 PM (Work)",
]

schedule_b = [
    "Friday 11/15: 7:30 PM - 9:00 PM (Video Call)",
    "Sunday 11/17: 6:00 PM - 7:00 PM (Yoga)",
]

location = "Downtown Brooklyn, New York"
weather_forecast = "Friday night: Clear, 45F. Saturday: Rain, 55F. Sunday: Cloudy, 50F."
vibe_request = "We want a romantic, fun date night this weekend. We're open to food, drinks, or an activity. Budget for the whole thing is $150."

# Combine the data into a single, structured prompt
prompt_data = f"""
PLANNING CONTEXT:
1. Couple's Availability (Busy Times):
   - Partner A: {'; '.join(schedule_a)}
   - Partner B: {'; '.join(schedule_b)}
2. Planning Window: Friday 11/15 through Sunday 11/17.
3. Location: {location}
4. Weather: {weather_forecast}
5. Vibe/Request: {vibe_request}
"""

In [9]:
plan_tool = {
    "name": "generate_date_plan",
    "description": "Generates a complete, optimized date night plan for a couple based on their schedules and preferences.",
    "input_schema": {
        "type": "object",
        "properties": {
            "date": {
                "type": "string",
                "description": "The determined optimal date and time (e.g., 'Saturday, 11/16 at 7:30 PM')."
            },
            "activity_type": {
                "type": "string",
                "description": "The category of the main activity (e.g., 'Restaurant', 'Activity', 'Bar')."
            },
            "name": {
                "type": "string",
                "description": "The specific name of the restaurant or venue (e.g., 'The Dapper Fox')."
            },
            "location_query": {
                "type": "string",
                "description": "A Google Maps search query to find the specific location (e.g., 'The Dapper Fox Downtown Brooklyn')."
            },
            "explanation": {
                "type": "string",
                "description": "A brief, compelling, and romantic explanation for why this is the perfect plan, including how it meets their constraints (schedules, weather, vibe)."
            }
        },
        "required": ["date", "activity_type", "name", "location_query", "explanation"]
    }
}

In [12]:
# System prompt guides the AI's behavior and tone
system_prompt = (
    "You are a world-class relationship planner and intelligent assistant. "
    "Your goal is to analyze the couple's constraints and romantic request to propose a single, "
    "perfectly optimized date night plan. You must use the provided `generate_date_plan` tool."
)

try:
    response = client.messages.create(
        model="claude-3-100k",
        max_tokens=2048,
        system=system_prompt,
        messages=[
            {"role": "user", "content": prompt_data}
        ],
        tools=[plan_tool]
    )

    # Check for the tool use block
    if response.stop_reason == "tool_use":
        tool_call = response.content[0]
        if tool_call.type == "tool_use":
            print("Claude's Optimal Date Plan:")
            # Nicely print the JSON structure
            print(json.dumps(tool_call.input, indent=4))
        else:
            print("Claude did not return a valid tool call.")
    else:
        print("Claude failed to produce a structured plan. Raw response:")
        print(response.content[0].text)

except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: Error code: 404 - {'type': 'error', 'error': {'type': 'not_found_error', 'message': 'model: claude-3-100k'}, 'request_id': 'req_011CUum6TXqqfWA4d4u7EEJV'}
